# Benchmark of existing approaches for detecting machine-generated text

## Contents 
1. [Solaiman](#Solaiman)
    1. [Install dependencies](#Install-dependencies)
    1. [Solaiman Code](#Solaiman-Code)
2. [Data Exploration](#Data-exploration)


## Solaiman

_Irene Solaiman, Miles Brundage, Jack Clark, Amanda Askell, Ariel Herbert-Voss, Jeff Wu, Alec Radford,
Gretchen Krueger, Jong Wook Kim, Sarah Kreps, Miles McCain, Alex Newhouse, Jason Blazakis, Kris McGuffie, and Jasmine Wang. 2019. Release strategies and the social impacts of language models._

### Install dependencies
Dependencies from: [https://github.com/HendrikStrobelt/detecting-fake-text/blob/master/requirements.txt](detecting-fake-text/requirements.txt).

### Solaiman Code
Source code: [https://github.com/openai/gpt-2-output-dataset/tree/master/detector](https://github.com/openai/gpt-2-output-dataset/tree/master/detector)

Logic extracted form `server.py`

In [2]:
import transformers
assert transformers.__version__ == '2.9.1', "Use transformers 2.9.1. It is available in the conda environment transf291"

/home/matteonardelli/miniconda3/envs/transf291/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import json
import torch
from urllib.parse import urlparse, unquote

#### Select model to use RoBERTa base or large

In [4]:
model_name = 'roberta-large'
#model_name = 'roberta-base'

#### Define model, tokenizer, and basic functions

In [5]:
model = RobertaForSequenceClassification.from_pretrained(model_name)
tokenizer = RobertaTokenizer.from_pretrained(model_name)
device='cuda' if torch.cuda.is_available() else 'cpu'

device = 'cpu'

In [6]:
def evaluate(query):
    tokens = tokenizer.encode(query)
    all_tokens = len(tokens)
    tokens = tokens[:tokenizer.max_len - 2]
    used_tokens = len(tokens)
    tokens = torch.tensor([tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id]).unsqueeze(0)
    mask = torch.ones_like(tokens)

    with torch.no_grad():
        logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
        probs = logits.softmax(dim=-1)

    fake, real = probs.detach().cpu().flatten().numpy().tolist()

    # Original: 
#    return json.dumps(dict(
#         all_tokens=all_tokens,
#         used_tokens=used_tokens,
#         real_probability=real,
#         fake_probability=fake
#     ))

# Changed to return only the binary classification result. 1 if sentence is likely machine-generated.
    return (fake > 0.5, fake, real)

def initialize(checkpoint):
#     if checkpoint.startswith('gs://'):
#         print(f'Downloading {checkpoint}', file=sys.stderr)
#         subprocess.check_output(['gsutil', 'cp', checkpoint, '.'])
#         checkpoint = os.path.basename(checkpoint)
#         assert os.path.isfile(checkpoint)

    print(f'Loading checkpoint from {checkpoint}')
    data = torch.load(checkpoint, map_location='cpu')
    model.load_state_dict(data['model_state_dict'])
    model.eval()

#### Download finetuned model and load it into RoBERTa

In [7]:
# !wget https://openaipublic.azureedge.net/gpt-2/detector-models/v1/detector-base.pt
# !wget https://openaipublic.azureedge.net/gpt-2/detector-models/v1/detector-large.pt

In [8]:
initialize('detector-large.pt')
# initialize('detector-base.pt')

Loading checkpoint from detector-large.pt


In [9]:
evaluate("hello world")

(True, 0.7026419639587402, 0.297357976436615)

## Data exploration

In [10]:
import json
import pandas as pd
import os
import time

In [11]:
data_path = "./../../get_text_detect_space/datasets/GPT2vsWebText/"
datasets = sorted([f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f))])

In [12]:
datasets

['.DS_Store',
 '175b_samples.jsonl',
 'large-762M-k40.test.jsonl',
 'large-762M-k40.train.jsonl',
 'large-762M-k40.valid.jsonl',
 'large-762M.test.jsonl',
 'large-762M.train.jsonl',
 'large-762M.valid.jsonl',
 'medium-345M-k40.test.jsonl',
 'medium-345M-k40.train.jsonl',
 'medium-345M-k40.valid.jsonl',
 'medium-345M.test.jsonl',
 'medium-345M.train.jsonl',
 'medium-345M.valid.jsonl',
 'small-117M-k40.test.jsonl',
 'small-117M-k40.train.jsonl',
 'small-117M-k40.valid.jsonl',
 'small-117M.test.jsonl',
 'small-117M.train.jsonl',
 'small-117M.valid.jsonl',
 'webtext.test.jsonl',
 'webtext.train.jsonl',
 'webtext.train.jsonl.clean100k.csv',
 'webtext.valid.jsonl',
 'xl-1542M-k40.test.jsonl',
 'xl-1542M-k40.train.jsonl',
 'xl-1542M-k40.valid.jsonl',
 'xl-1542M.test.jsonl',
 'xl-1542M.train.jsonl',
 'xl-1542M.train.jsonl.clean100k.csv',
 'xl-1542M.valid.jsonl']

### Human-written data

In [13]:
# ds_hw_filename = 'webtext.train.jsonl'
# ds_hw = pd.read_json(os.path.join(data_path, ds_hw_filename), lines = True)     

ds_hw_filename = 'webtext.train.jsonl.clean100k.csv'
ds_hw = pd.read_csv(os.path.join(data_path, ds_hw_filename))  

# with open(os.path.join(data_path, ds_hw_filename), 'r') as file:
#    ds_hw = [x for x in file.readlines() if x is not None and x != "\n"]     

In [14]:
ds_hw

,id,ended,length,text,english words,english %,non-eng vocabs
0,172092,True,634,Everton goalkeeper Tim Howard has signed a con...,488,95,7
1,208655,True,264,That addresses performance. As for battery lif...,202,97,1
2,25537,False,1024,"Posted on February 18, 2014 by Bryan Ball\n\nB...",819,97,6
3,81025,True,335,Jasper Jolly\n\nhe US dollar has reached its h...,266,97,5
4,6966,True,48,I received a package containing food contents ...,41,100,0
...,...,...,...,...,...,...,...
99995,22089,False,1024,Who Pays\n\nAmbulance Billing\n\nThe Ministry ...,785,100,0
99996,146668,True,466,Description\n\nAutomatic Bottle Opener\n\n\nTh...,366,98,5
99997,100597,True,89,"We have created a bunch of 'how-to' videos, to...",66,95,2
99998,64251,True,72,$14.99\n\nEasily & securely mount your iPhone ...,42,95,2


In [15]:
start = time.time()
payload = evaluate(ds_hw.iloc[20].text)
# payload = evaluate(ds_hw[20])
end = time.time()
print("{:.2f} Seconds for a check with GPT-2".format(end - start))
print(f"machine generated? {payload}")

0.27 Seconds for a check with GPT-2
machine generated? (False, 0.000952672678977251, 0.9990473389625549)


### Machine-generated data

In [16]:
# ds_mg_filename = 'small-117M.train.jsonl'
# ds_mg = pd.read_json(os.path.join(data_path, ds_mg_filename), lines = True)  
ds_mg_filename = 'xl-1542M.train.jsonl.clean100k.csv'
ds_mg = pd.read_csv(os.path.join(data_path, ds_mg_filename))  
# 
# with open(os.path.join(data_path, ds_mg_filename), 'r') as file:
#    ds_mg = [x for x in file.readlines() if x is not None and x != "\n"]

In [17]:
ds_mg

,id,text,length,ended,english words,english %,non-eng vocabs
0,140132,Dr Dollemayne said he was ever mindful of the ...,682,True,529,97,8
1,43198,James Raymond is among the authors whose artic...,1024,False,715,95,35
2,58415,ZANSKY PORTSMOUTH in Belfast admitted download...,747,True,582,96,18
3,223826,A relief that has become the byword for writer...,1024,False,789,98,14
4,206739,$78.00 / Sold Out\n\n(SOLD OUT)\n\nCompared to...,504,True,355,95,11
...,...,...,...,...,...,...,...
99995,59284,Install with only 1 click! Removes the first p...,39,True,28,96,1
99996,80292,Riun Davis named the starting quarterback of t...,884,True,681,97,14
99997,199811,Amtrak train derailed in Oklahoma after curtai...,711,True,571,98,6
99998,19150,"Gary Caradori, The Associated Press\n\n\nCARAC...",245,True,179,95,7


In [19]:
start = time.time()
payload = evaluate(ds_mg.iloc[20].text)
# payload = evaluate(ds_mg[20])
end = time.time()
print("{:.2f} Seconds for a check with GPT-2".format(end - start))
print(f"machine generated? {payload}")

Token indices sequence length is longer than the specified maximum sequence length for this model (1024 > 512). Running this sequence through the model will result in indexing errors


0.53 Seconds for a check with GPT-2
machine generated? (True, 0.9997244477272034, 0.0002755813766270876)


## Evaluation
### Auxiliary functions

In [18]:
import numpy as np
from sklearn.utils import shuffle

def create_dataset(raw_data, is_machine_generated=True):
    X = np.array(raw_data)
    if is_machine_generated:
        y = np.ones_like(X, dtype=int)
    else:
        y = np.zeros_like(X, dtype=int)
    return pd.DataFrame(data={'X':X, 'y':y})

def concat_and_shuffle(datasets):
    return shuffle(pd.concat(datasets)).reset_index()

In [19]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

def safe_macro_f1(y, y_pred):
    """
    Macro-averaged F1, forcing `sklearn` to report as a multiclass
    problem even when there are just two classes. `y` is the list of
    gold labels and `y_pred` is the list of predicted labels.

    """
    return f1_score(y, y_pred, average='macro', pos_label=None)

def safe_accuracy(y, y_pred):
    return accuracy_score(y, y_pred, normalize=True)

def gtc_evaluate(
        dataset,
        model,
        score_func=safe_accuracy):

    # Predictions if we have labels:
    y_ds, preds = model.predict(dataset)
    confusion = confusion_matrix(y_ds, preds)
    score = score_func(y_ds, preds)
        
    # Return the overall scores and other experimental info:
    return {
        'model': model,
#        'labeled_outputs': y_ds, 
#        'predictions': preds, 
        'confusion_matrix': confusion, 
        'score': score }


In [36]:
class Wrapper:
    def __init__(self):
        pass

    def predict(self, dataset, truncateAt=512):
        y_pred = []
        y_dataset = []
        
        totalsize = len(dataset)
        progressupdate = max(min(100, int(totalsize/10)), 1)
        prog = 0
        print(f"progupd: {progressupdate}; m(m(100, {int(totalsize/10)}), 1)")
        for index, row in dataset.iterrows():
            prog+=1
            if not row['X']: 
                print("Skipping sentence because it appears to be empty")
                continue
            try:
                truncated_sentence = row['X'][:truncateAt] if len(row['X']) > truncateAt else row['X']
                (is_generated, fake, real) = evaluate(truncated_sentence)
                y_pred.append(1 if is_generated else 0)     
                y_dataset.append(row['y'])
                
                if (index) % progressupdate == 0:
                    print(f"... progress: {index} ({prog / totalsize})")
            except:
                print(f"Error while processing the query: {row['X']}")
                break
        return (y_dataset, y_pred)

In [37]:
solaiman_detector = Wrapper()

### Playing around

In [22]:
# mg_pd = create_dataset(ds_mg["text"], is_machine_generated=True)
# hw_pd = create_dataset(ds_hw["text"], is_machine_generated=False)
# shuffled = concat_and_shuffle([mg_pd, hw_pd])
# sample_df = shuffled.sample(n=10, random_state=1)

# creating dataset from clean100k csv files
mg_pd = create_dataset(ds_mg["text"], is_machine_generated=True)
hw_pd = create_dataset(ds_hw["text"], is_machine_generated=False)

In [23]:
len(mg_pd)

100000

Testing whether the `gtc_evaluate()` works as expected. By default, we use accuracy to measure the score.

In [24]:
def gtc_evaluate_with_checkpoints(dataset, 
        model,
        checkpoint = 10000,
        score_func=safe_accuracy):
    length = len(dataset)
    splits = min(int(length/checkpoint), length) 
    confmatrix_sum = np.zeros((2, 2))
    for i in range(splits + 1):
        dataset_split = dataset[i * checkpoint : (i+1)*checkpoint]
        if dataset_split.empty:
            break
        partial_r = gtc_evaluate(dataset_split, solaiman_detector, score_func=score_func)
        print (f"----- split #{i} (size: {len(dataset_split)}) -----\n {partial_r['confusion_matrix']}\n----- acc: {partial_r['score']} -----")
        confmatrix_sum += partial_r['confusion_matrix']
    print(f"====================================")
    print(f" Final confusion matrix:\n {confmatrix_sum}")
    (tn, fp, fn, tp) = confmatrix_sum.ravel()
    accuracy = (tp +tn) / (tp + tn + fp + fn)
    print(f" Final accuracy: {accuracy}")
    print(f"====================================")

In [ ]:
%%time
gtc_evaluate_with_checkpoints(mg_pd, solaiman_detector, checkpoint=10000, score_func=safe_accuracy)

In [ ]:
# %%time
# gtc_evaluate(mg_pd[:10], solaiman_detector, score_func=safe_accuracy)

The Solaiman's model warns that sentences should have less than 512 tokens. So, let's try by filtering out senteces with more tokens than that.

### experiments on raw data

In [25]:
ds_hwraw_filename = 'webtext.test.jsonl'
ds_hw_raw = pd.read_json(os.path.join(data_path, ds_hwraw_filename), lines = True)

In [26]:
ds_mgraw_filename = 'xl-1542M.test.jsonl'
ds_mg_raw = pd.read_json(os.path.join(data_path, ds_mgraw_filename), lines = True)

In [27]:
ds_hw_raw

,id,ended,length,text
0,255000,True,134,Is this restaurant family-friendly ? Yes No Un...
1,255001,True,713,Clinton talks about her time of 'reflection' d...
2,255002,True,173,House Majority Whip Steve Scalise has been dis...
3,255003,True,490,Insight Course: Lesson 14\n\nControl of the Mi...
4,255004,True,220,BY JENNIE MCNULTY\n\nLesbian.com\n\nYou know W...
...,...,...,...,...
4995,259995,False,1024,"The week after my 30th birthday, my best frien..."
4996,259996,True,300,Downton Abbey doesn't return for its third sea...
4997,259997,True,360,Amazon Studios said today that Stephen Root ha...
4998,259998,True,607,Everyone likes clean wheels but not everyone h...


In [39]:
ds_mg_raw

,id,text,length,ended
0,255001,What is this?\n\nThis is a neat little plugin ...,162,True
1,255002,People carry a memorial sign to honor Robert P...,639,True
2,255003,Google is launching a new scholarship campaign...,434,True
3,255004,"Authored by: Kristin Scavuzzo, Managing Direct...",1024,False
4,255005,>at some point the MPP team stop standardizing...,1024,False
...,...,...,...,...
4995,259996,Mac coverage beginning in 1926 at The Michigan...,68,True
4996,259997,Martyr Dadabhoyao got a big experience about s...,362,True
4997,259998,"""The nation's smugglers have changed their tac...",1024,False
4998,259999,The level of British public support for direct...,644,True


In [40]:
mg_raw_pd = create_dataset(ds_mg_raw["text"], is_machine_generated=True)
hw_raw_pd = create_dataset(ds_hw_raw["text"], is_machine_generated=False)

In [41]:
mg_raw_pd

,X,y
0,What is this?\n\nThis is a neat little plugin ...,1
1,People carry a memorial sign to honor Robert P...,1
2,Google is launching a new scholarship campaign...,1
3,"Authored by: Kristin Scavuzzo, Managing Direct...",1
4,>at some point the MPP team stop standardizing...,1
...,...,...
4995,Mac coverage beginning in 1926 at The Michigan...,1
4996,Martyr Dadabhoyao got a big experience about s...,1
4997,"""The nation's smugglers have changed their tac...",1
4998,The level of British public support for direct...,1


In [42]:
hw_raw_pd

,X,y
0,Is this restaurant family-friendly ? Yes No Un...,0
1,Clinton talks about her time of 'reflection' d...,0
2,House Majority Whip Steve Scalise has been dis...,0
3,Insight Course: Lesson 14\n\nControl of the Mi...,0
4,BY JENNIE MCNULTY\n\nLesbian.com\n\nYou know W...,0
...,...,...
4995,"The week after my 30th birthday, my best frien...",0
4996,Downton Abbey doesn't return for its third sea...,0
4997,Amazon Studios said today that Stephen Root ha...,0
4998,Everyone likes clean wheels but not everyone h...,0


In [43]:
%%time
gtc_evaluate_with_checkpoints(mg_raw_pd, solaiman_detector, checkpoint=1000, score_func=safe_accuracy)

progupd: 100; m(m(100, 100), 1)
... progress: 0 (0.001)
... progress: 100 (0.101)
... progress: 200 (0.201)
... progress: 300 (0.301)
... progress: 400 (0.401)
... progress: 500 (0.501)
... progress: 600 (0.601)
... progress: 700 (0.701)
... progress: 800 (0.801)
... progress: 900 (0.901)
----- split #0 (size: 1000) -----
 [[  0   0]
 [132 868]]
----- acc: 0.868 -----
progupd: 100; m(m(100, 100), 1)
... progress: 1000 (0.001)
... progress: 1100 (0.101)
... progress: 1200 (0.201)
... progress: 1300 (0.301)
... progress: 1400 (0.401)
... progress: 1500 (0.501)
... progress: 1600 (0.601)
... progress: 1700 (0.701)
... progress: 1800 (0.801)
... progress: 1900 (0.901)
----- split #1 (size: 1000) -----
 [[  0   0]
 [124 876]]
----- acc: 0.876 -----
progupd: 100; m(m(100, 100), 1)
... progress: 2000 (0.001)
... progress: 2100 (0.101)
... progress: 2200 (0.201)
... progress: 2300 (0.301)
... progress: 2400 (0.401)
... progress: 2500 (0.501)
... progress: 2600 (0.601)
... progress: 2700 (0.701

In [44]:
%%time
gtc_evaluate_with_checkpoints(hw_raw_pd, solaiman_detector, checkpoint=1000, score_func=safe_accuracy)

progupd: 100; m(m(100, 100), 1)
... progress: 0 (0.001)
... progress: 100 (0.101)
... progress: 200 (0.201)
... progress: 300 (0.301)
... progress: 400 (0.401)
... progress: 500 (0.501)
... progress: 600 (0.601)
... progress: 700 (0.701)
... progress: 800 (0.801)
... progress: 900 (0.901)
----- split #0 (size: 1000) -----
 [[955  45]
 [  0   0]]
----- acc: 0.955 -----
progupd: 100; m(m(100, 100), 1)
... progress: 1000 (0.001)
... progress: 1100 (0.101)
... progress: 1200 (0.201)
... progress: 1300 (0.301)
... progress: 1400 (0.401)
... progress: 1500 (0.501)
... progress: 1600 (0.601)
... progress: 1700 (0.701)
... progress: 1800 (0.801)
... progress: 1900 (0.901)
----- split #1 (size: 1000) -----
 [[947  53]
 [  0   0]]
----- acc: 0.947 -----
progupd: 100; m(m(100, 100), 1)
... progress: 2000 (0.001)
... progress: 2100 (0.101)
... progress: 2200 (0.201)
... progress: 2300 (0.301)
... progress: 2400 (0.401)
... progress: 2500 (0.501)
... progress: 2600 (0.601)
... progress: 2700 (0.701

### GPT-3 Machine generated text

In [57]:
gpt3_data_path = "./../../get_text_detect_space/datasets/GPT3"
gpt3_ds_mg_filename = '175b_samples.jsonl'
with open(os.path.join(gpt3_data_path, gpt3_ds_mg_filename), 'r') as file:
    gpt3_ds_mg = [x for x in file.readlines() if x is not None and x != "\n"]

In [58]:
gpt3_mg_pd = create_dataset(gpt3_ds_mg, is_machine_generated=True)

In [59]:
gpt3_mg_pd

,X,y
0,"""Glacier Ridge Christian School\n\nGlacier Rid...",1
1,"""His father was a professor and his mother was...",1
2,"""Image caption Bailiffs removed protesters fro...",1
3,"""Contents\n\n1. The Meaning of Love\n\n2. Our ...",1
4,"""Interviews\n\nInterview with Ariel Efraim (So...",1
...,...,...
480,"""Get Rangers updates directly to your inbox Su...",1
481,"""Posted 01 September 2010 - 07:51 PM\n\nUsing ...",1
482,"""Jeff Chapman\n\nJeffrey Scott Chapman (born M...",1
483,"""You can look into the Issue Tracker and repor...",1


In [61]:
%%time
gtc_evaluate_with_checkpoints(gpt3_mg_pd, solaiman_detector, checkpoint=1000, score_func=safe_accuracy)

progupd: 4; m(m(100, 4), 1)
... progress: 0 (0.002061855670103093)
... progress: 4 (0.010309278350515464)
... progress: 8 (0.018556701030927835)
... progress: 12 (0.026804123711340205)
... progress: 16 (0.03505154639175258)
... progress: 20 (0.04329896907216495)
... progress: 24 (0.05154639175257732)
... progress: 28 (0.05979381443298969)
... progress: 32 (0.06804123711340206)
... progress: 36 (0.07628865979381444)
... progress: 40 (0.08453608247422681)
... progress: 44 (0.09278350515463918)
... progress: 48 (0.10103092783505155)
... progress: 52 (0.10927835051546392)
... progress: 56 (0.11752577319587629)
... progress: 60 (0.12577319587628866)
... progress: 64 (0.13402061855670103)
... progress: 68 (0.1422680412371134)
... progress: 72 (0.15051546391752577)
... progress: 76 (0.15876288659793814)
... progress: 80 (0.1670103092783505)
... progress: 84 (0.17525773195876287)
... progress: 88 (0.18350515463917524)
... progress: 92 (0.19175257731958764)
... progress: 96 (0.2)
... progress: 

Token indices sequence length is longer than the specified maximum sequence length for this model (1026 > 512). Running this sequence through the model will result in indexing errors


... progress: 188 (0.38969072164948454)
... progress: 192 (0.3979381443298969)
... progress: 196 (0.4061855670103093)
... progress: 200 (0.4144329896907217)
... progress: 204 (0.422680412371134)
... progress: 208 (0.4309278350515464)
... progress: 212 (0.43917525773195876)
... progress: 216 (0.44742268041237115)
... progress: 220 (0.4556701030927835)
... progress: 224 (0.4639175257731959)
... progress: 228 (0.47216494845360824)
... progress: 232 (0.48041237113402063)
... progress: 236 (0.488659793814433)
... progress: 240 (0.49690721649484537)
... progress: 244 (0.5051546391752577)
... progress: 248 (0.51340206185567)
... progress: 252 (0.5216494845360825)
... progress: 256 (0.5298969072164949)
... progress: 260 (0.5381443298969072)
... progress: 264 (0.5463917525773195)
... progress: 268 (0.554639175257732)
... progress: 272 (0.5628865979381443)
... progress: 276 (0.5711340206185567)
... progress: 280 (0.5793814432989691)
... progress: 284 (0.5876288659793815)
... progress: 288 (0.595

### GROVER dataset

In [43]:
grover_data_path = "./../../get_text_detect_space/datasets/GROVER/"
grover_datasets = sorted([f for f in os.listdir(grover_data_path) if os.path.isfile(os.path.join(grover_data_path, f))])

In [44]:
grover_datasets

['generator=base~dataset=p0.90.jsonl',
 'generator=base~dataset=p0.92.jsonl',
 'generator=base~dataset=p0.94.jsonl',
 'generator=base~dataset=p0.96.jsonl',
 'generator=base~dataset=p0.98.jsonl',
 'generator=base~dataset=p1.00.jsonl',
 'generator=medium~dataset=p0.90.jsonl',
 'generator=medium~dataset=p0.92.jsonl',
 'generator=medium~dataset=p0.94.jsonl',
 'generator=medium~dataset=p0.96.jsonl',
 'generator=medium~dataset=p0.98.jsonl',
 'generator=medium~dataset=p1.00.jsonl',
 'generator=mega~dataset=p0.90.jsonl',
 'generator=mega~dataset=p0.92.jsonl',
 'generator=mega~dataset=p0.94.jsonl',
 'generator=mega~dataset=p0.96.jsonl',
 'generator=mega~dataset=p0.98.jsonl',
 'generator=mega~dataset=p1.00.jsonl']

In [47]:
grover_ds_filename = 'generator=mega~dataset=p1.00.jsonl'
grover_ds = pd.read_json(os.path.join(grover_data_path, grover_ds_filename), lines = True)

In [48]:
grover_ds_mg = grover_ds[(grover_ds['label'] == 'machine')]
grover_ds_hw = grover_ds[(grover_ds['label'] == 'human')]

In [49]:
grover_ds_hw

,article,domain,title,date,authors,ind30k,url,label,orig_split,split,random_score,top_p
0,Patrons crowd the platform at the Washington M...,theepochtimes.com,US Senators Threaten Metro Funding Over Chines...,2019-04-15,None,29363,https://www.theepochtimes.com/us-senators-thre...,human,train_burner,train,-4.263557,NaN
2,In December 2018 the issue of why the Canadian...,ottawacitizen.com,Military assessment ruling out the need for a ...,2019-04-16,Updated,23090,https://ottawacitizen.com/news/national/defenc...,human,train_burner,train,-3.863624,NaN
3,Modi also said the opposition wants to strip t...,deccanchronicle.com,Mahamilavat gang scared I will shut their shop...,2019-04-11,None,28881,https://www.deccanchronicle.com/nation/politic...,human,train_burner,train,-3.813725,NaN
6,Warning: the night is dark and full of spoiler...,popsugar.com,Who Are the Three Heads of the Dragon in Game ...,2019-04-20,Quinn Keaney,21352,https://www.popsugar.com/entertainment/Who-Thr...,human,train_burner,train,-3.482009,NaN
8,"AUGUSTA, Ga. — Two players expected to be in t...",nypost.com,It was a forgettable day for Jordan Spieth and...,2019-04-12,None,25122,https://nypost.com/2019/04/12/it-was-a-forgett...,human,train_burner,train,-3.214130,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
24992,"“High Fidelity,” an upcoming series starring Z...",variety.com,Zoe Kravitz Series ‘High Fidelity’ Moves to Hu...,2019-04-09,"Will Thorne, Staff Writer, Will'S Most Recent ...",28239,https://variety.com/2019/tv/news/zoe-kravitz-s...,human,train_burner,test,3.004529,NaN
24993,Service changes for the real L train “shutdown...,amny.com,Service suspended on L train between Manhattan...,2019-04-15,Lauren Cook,2904,https://www.amny.com/transit/l-train-overnight...,human,gen,test,3.029303,NaN
24995,President Donald Trump is firing the director ...,cyprus-mail.com,Trump orders firing of US Secret Service chief,2019-04-08,Reuters News Service,4677,https://cyprus-mail.com/2019/04/08/trump-order...,human,gen,test,3.049210,NaN
24997,Spread the love\nTesla raising price of its fu...,mangalorean.com,Tesla raising price of its full self-driving o...,2019-04-15,Please Enter Your Name Here,26771,http://www.mangalorean.com/tesla-raising-price...,human,train_burner,test,3.167816,NaN


In [50]:
grover_mg_pd = create_dataset(grover_ds_mg["article"], is_machine_generated=True)
grover_hw_pd = create_dataset(grover_ds_hw["article"], is_machine_generated=False)

In [51]:
grover_hw_pd

,X,y
0,Patrons crowd the platform at the Washington M...,0
1,In December 2018 the issue of why the Canadian...,0
2,Modi also said the opposition wants to strip t...,0
3,Warning: the night is dark and full of spoiler...,0
4,"AUGUSTA, Ga. — Two players expected to be in t...",0
...,...,...
14995,"“High Fidelity,” an upcoming series starring Z...",0
14996,Service changes for the real L train “shutdown...,0
14997,President Donald Trump is firing the director ...,0
14998,Spread the love\nTesla raising price of its fu...,0


#### Evaluate

In [89]:
%%time
gtc_evaluate_with_checkpoints(grover_mg_pd, solaiman_detector, checkpoint=2500, score_func=safe_accuracy)

progupd: 25; m(m(100, 25), 1)
... progress: 0 (0.0004)
... progress: 25 (0.0104)
... progress: 50 (0.0204)
... progress: 75 (0.0304)
... progress: 100 (0.0404)
... progress: 125 (0.0504)
... progress: 150 (0.0604)
... progress: 175 (0.0704)
... progress: 200 (0.0804)
... progress: 225 (0.0904)
... progress: 250 (0.1004)
... progress: 275 (0.1104)
... progress: 300 (0.1204)
... progress: 325 (0.1304)
... progress: 350 (0.1404)
... progress: 375 (0.1504)
... progress: 400 (0.1604)
... progress: 425 (0.1704)
... progress: 450 (0.1804)
... progress: 475 (0.1904)
... progress: 500 (0.2004)
... progress: 525 (0.2104)
... progress: 550 (0.2204)
... progress: 575 (0.2304)
... progress: 600 (0.2404)
... progress: 625 (0.2504)
... progress: 650 (0.2604)
... progress: 675 (0.2704)
... progress: 700 (0.2804)
... progress: 725 (0.2904)
... progress: 750 (0.3004)
... progress: 775 (0.3104)
... progress: 800 (0.3204)
... progress: 825 (0.3304)
... progress: 850 (0.3404)
... progress: 875 (0.3504)
...

... progress: 7125 (0.8504)
... progress: 7150 (0.8604)
... progress: 7175 (0.8704)
... progress: 7200 (0.8804)
... progress: 7225 (0.8904)
... progress: 7250 (0.9004)
... progress: 7275 (0.9104)
... progress: 7300 (0.9204)
... progress: 7325 (0.9304)
... progress: 7350 (0.9404)
... progress: 7375 (0.9504)
... progress: 7400 (0.9604)
... progress: 7425 (0.9704)
... progress: 7450 (0.9804)
... progress: 7475 (0.9904)
----- split #2 (size: 2500) -----
 [[   0    0]
 [ 977 1523]]
----- acc: 0.6092 -----
progupd: 25; m(m(100, 25), 1)
... progress: 7500 (0.0004)
... progress: 7525 (0.0104)
... progress: 7550 (0.0204)
... progress: 7575 (0.0304)
... progress: 7600 (0.0404)
... progress: 7625 (0.0504)
... progress: 7650 (0.0604)
... progress: 7675 (0.0704)
... progress: 7700 (0.0804)
... progress: 7725 (0.0904)
... progress: 7750 (0.1004)
... progress: 7775 (0.1104)
... progress: 7800 (0.1204)
... progress: 7825 (0.1304)
... progress: 7850 (0.1404)
... progress: 7875 (0.1504)
... progress: 79

In [90]:
%%time
gtc_evaluate_with_checkpoints(grover_hw_pd, solaiman_detector, checkpoint=2500, score_func=safe_accuracy)

progupd: 25; m(m(100, 25), 1)
... progress: 0 (0.0004)
... progress: 25 (0.0104)
... progress: 50 (0.0204)
... progress: 75 (0.0304)
... progress: 100 (0.0404)
... progress: 125 (0.0504)
... progress: 150 (0.0604)
... progress: 175 (0.0704)
... progress: 200 (0.0804)
... progress: 225 (0.0904)
... progress: 250 (0.1004)
... progress: 275 (0.1104)
... progress: 300 (0.1204)
... progress: 325 (0.1304)
... progress: 350 (0.1404)
... progress: 375 (0.1504)
... progress: 400 (0.1604)
... progress: 425 (0.1704)
... progress: 450 (0.1804)
... progress: 475 (0.1904)
... progress: 500 (0.2004)
... progress: 525 (0.2104)
... progress: 550 (0.2204)
... progress: 575 (0.2304)
... progress: 600 (0.2404)
... progress: 625 (0.2504)
... progress: 650 (0.2604)
... progress: 675 (0.2704)
... progress: 700 (0.2804)
... progress: 725 (0.2904)
... progress: 750 (0.3004)
... progress: 775 (0.3104)
... progress: 800 (0.3204)
... progress: 825 (0.3304)
... progress: 850 (0.3404)
... progress: 875 (0.3504)
...

... progress: 7125 (0.8504)
... progress: 7150 (0.8604)
... progress: 7175 (0.8704)
... progress: 7200 (0.8804)
... progress: 7225 (0.8904)
... progress: 7250 (0.9004)
... progress: 7275 (0.9104)
... progress: 7300 (0.9204)
... progress: 7325 (0.9304)
... progress: 7350 (0.9404)
... progress: 7375 (0.9504)
... progress: 7400 (0.9604)
... progress: 7425 (0.9704)
... progress: 7450 (0.9804)
... progress: 7475 (0.9904)
----- split #2 (size: 2500) -----
 [[2415   85]
 [   0    0]]
----- acc: 0.966 -----
progupd: 25; m(m(100, 25), 1)
... progress: 7500 (0.0004)
... progress: 7525 (0.0104)
... progress: 7550 (0.0204)
... progress: 7575 (0.0304)
... progress: 7600 (0.0404)
... progress: 7625 (0.0504)
... progress: 7650 (0.0604)
... progress: 7675 (0.0704)
... progress: 7700 (0.0804)
... progress: 7725 (0.0904)
... progress: 7750 (0.1004)
... progress: 7775 (0.1104)
... progress: 7800 (0.1204)
... progress: 7825 (0.1304)
... progress: 7850 (0.1404)
... progress: 7875 (0.1504)
... progress: 790

... progress: 14000 (0.6004)
... progress: 14025 (0.6104)
... progress: 14050 (0.6204)
... progress: 14075 (0.6304)
... progress: 14100 (0.6404)
... progress: 14125 (0.6504)
... progress: 14150 (0.6604)
... progress: 14175 (0.6704)
... progress: 14200 (0.6804)
... progress: 14225 (0.6904)
... progress: 14250 (0.7004)
... progress: 14275 (0.7104)
... progress: 14300 (0.7204)
... progress: 14325 (0.7304)
... progress: 14350 (0.7404)
... progress: 14375 (0.7504)
... progress: 14400 (0.7604)
... progress: 14425 (0.7704)
... progress: 14450 (0.7804)
... progress: 14475 (0.7904)
... progress: 14500 (0.8004)
... progress: 14525 (0.8104)
... progress: 14550 (0.8204)
... progress: 14575 (0.8304)
... progress: 14600 (0.8404)
... progress: 14625 (0.8504)
... progress: 14650 (0.8604)
... progress: 14675 (0.8704)
... progress: 14700 (0.8804)
... progress: 14725 (0.8904)
... progress: 14750 (0.9004)
... progress: 14775 (0.9104)
... progress: 14800 (0.9204)
... progress: 14825 (0.9304)
... progress: 

#### p90

In [54]:
grover_ds_filename = 'generator=mega~dataset=p0.90.jsonl'
grover_ds = pd.read_json(os.path.join(grover_data_path, grover_ds_filename), lines = True)
grover_ds_mg = grover_ds[(grover_ds['label'] == 'machine')]
grover_ds_hw = grover_ds[(grover_ds['label'] == 'human')]
grover_mg_pd = create_dataset(grover_ds_mg["article"], is_machine_generated=True)
grover_hw_pd = create_dataset(grover_ds_hw["article"], is_machine_generated=False)

In [55]:
grover_hw_pd

,X,y
0,Patrons crowd the platform at the Washington M...,0
1,In December 2018 the issue of why the Canadian...,0
2,Modi also said the opposition wants to strip t...,0
3,Warning: the night is dark and full of spoiler...,0
4,"AUGUSTA, Ga. — Two players expected to be in t...",0
...,...,...
14995,"“High Fidelity,” an upcoming series starring Z...",0
14996,Service changes for the real L train “shutdown...,0
14997,President Donald Trump is firing the director ...,0
14998,Spread the love\nTesla raising price of its fu...,0


In [95]:
%%time
gtc_evaluate_with_checkpoints(grover_hw_pd, solaiman_detector, checkpoint=2500, score_func=safe_accuracy)

progupd: 25; m(m(100, 25), 1)
... progress: 0 (0.0004)
... progress: 25 (0.0104)
... progress: 50 (0.0204)
... progress: 75 (0.0304)
... progress: 100 (0.0404)
... progress: 125 (0.0504)
... progress: 150 (0.0604)
... progress: 175 (0.0704)
... progress: 200 (0.0804)
... progress: 225 (0.0904)
... progress: 250 (0.1004)
... progress: 275 (0.1104)
... progress: 300 (0.1204)
... progress: 325 (0.1304)
... progress: 350 (0.1404)
... progress: 375 (0.1504)
... progress: 400 (0.1604)
... progress: 425 (0.1704)
... progress: 450 (0.1804)
... progress: 475 (0.1904)
... progress: 500 (0.2004)
... progress: 525 (0.2104)
... progress: 550 (0.2204)
... progress: 575 (0.2304)
... progress: 600 (0.2404)
... progress: 625 (0.2504)
... progress: 650 (0.2604)
... progress: 675 (0.2704)
... progress: 700 (0.2804)
... progress: 725 (0.2904)
... progress: 750 (0.3004)
... progress: 775 (0.3104)
... progress: 800 (0.3204)
... progress: 825 (0.3304)
... progress: 850 (0.3404)
... progress: 875 (0.3504)
...

... progress: 7125 (0.8504)
... progress: 7150 (0.8604)
... progress: 7175 (0.8704)
... progress: 7200 (0.8804)
... progress: 7225 (0.8904)
... progress: 7250 (0.9004)
... progress: 7275 (0.9104)
... progress: 7300 (0.9204)
... progress: 7325 (0.9304)
... progress: 7350 (0.9404)
... progress: 7375 (0.9504)
... progress: 7400 (0.9604)
... progress: 7425 (0.9704)
... progress: 7450 (0.9804)
... progress: 7475 (0.9904)
----- split #2 (size: 2500) -----
 [[2415   85]
 [   0    0]]
----- acc: 0.966 -----
progupd: 25; m(m(100, 25), 1)
... progress: 7500 (0.0004)
... progress: 7525 (0.0104)
... progress: 7550 (0.0204)
... progress: 7575 (0.0304)
... progress: 7600 (0.0404)
... progress: 7625 (0.0504)
... progress: 7650 (0.0604)
... progress: 7675 (0.0704)
... progress: 7700 (0.0804)
... progress: 7725 (0.0904)
... progress: 7750 (0.1004)
... progress: 7775 (0.1104)
... progress: 7800 (0.1204)
... progress: 7825 (0.1304)
... progress: 7850 (0.1404)
... progress: 7875 (0.1504)
... progress: 790

... progress: 14000 (0.6004)
... progress: 14025 (0.6104)
... progress: 14050 (0.6204)
... progress: 14075 (0.6304)
... progress: 14100 (0.6404)
... progress: 14125 (0.6504)
... progress: 14150 (0.6604)
... progress: 14175 (0.6704)
... progress: 14200 (0.6804)
... progress: 14225 (0.6904)
... progress: 14250 (0.7004)
... progress: 14275 (0.7104)
... progress: 14300 (0.7204)
... progress: 14325 (0.7304)
... progress: 14350 (0.7404)
... progress: 14375 (0.7504)
... progress: 14400 (0.7604)
... progress: 14425 (0.7704)
... progress: 14450 (0.7804)
... progress: 14475 (0.7904)
... progress: 14500 (0.8004)
... progress: 14525 (0.8104)
... progress: 14550 (0.8204)
... progress: 14575 (0.8304)
... progress: 14600 (0.8404)
... progress: 14625 (0.8504)
... progress: 14650 (0.8604)
... progress: 14675 (0.8704)
... progress: 14700 (0.8804)
... progress: 14725 (0.8904)
... progress: 14750 (0.9004)
... progress: 14775 (0.9104)
... progress: 14800 (0.9204)
... progress: 14825 (0.9304)
... progress: 

In [56]:
grover_mg_pd

,X,y
0,As the price of oil continues to rise due to i...,1
1,Following a community meeting held yesterday a...,1
2,"news, local-news,\nMore than 170 submissions w...",1
3,Danny Drinkwater was a player for Bournemouth ...,1
4,"Or, more like, more like ""it's gravy-time!"" Fo...",1
...,...,...
9995,FILE PHOTO: Syria's President Bashar al-Assad'...,1
9996,Who wouldn’t want a perfectly square dinner pl...,1
9997,After an emergency overnight closure of the L ...,1
9998,US President Donald Trump said on Monday he ha...,1


In [57]:
%%time
gtc_evaluate_with_checkpoints(grover_mg_pd, solaiman_detector, checkpoint=2500, score_func=safe_accuracy)

progupd: 25; m(m(100, 25), 1)
... progress: 0 (0.0004)
... progress: 25 (0.0104)
... progress: 50 (0.0204)
... progress: 75 (0.0304)
... progress: 100 (0.0404)
... progress: 125 (0.0504)
... progress: 150 (0.0604)
... progress: 175 (0.0704)
... progress: 200 (0.0804)
... progress: 225 (0.0904)
... progress: 250 (0.1004)
... progress: 275 (0.1104)
... progress: 300 (0.1204)
... progress: 325 (0.1304)
... progress: 350 (0.1404)
... progress: 375 (0.1504)
... progress: 400 (0.1604)
... progress: 425 (0.1704)
... progress: 450 (0.1804)
... progress: 475 (0.1904)
... progress: 500 (0.2004)
... progress: 525 (0.2104)
... progress: 550 (0.2204)
... progress: 575 (0.2304)
... progress: 600 (0.2404)
... progress: 625 (0.2504)
... progress: 650 (0.2604)
... progress: 675 (0.2704)
... progress: 700 (0.2804)
... progress: 725 (0.2904)
... progress: 750 (0.3004)
... progress: 775 (0.3104)
... progress: 800 (0.3204)
... progress: 825 (0.3304)
... progress: 850 (0.3404)
... progress: 875 (0.3504)
...

... progress: 7125 (0.8504)
... progress: 7150 (0.8604)
... progress: 7175 (0.8704)
... progress: 7200 (0.8804)
... progress: 7225 (0.8904)
... progress: 7250 (0.9004)
... progress: 7275 (0.9104)
... progress: 7300 (0.9204)
... progress: 7325 (0.9304)
... progress: 7350 (0.9404)
... progress: 7375 (0.9504)
... progress: 7400 (0.9604)
... progress: 7425 (0.9704)
... progress: 7450 (0.9804)
... progress: 7475 (0.9904)
----- split #2 (size: 2500) -----
 [[   0    0]
 [1095 1405]]
----- acc: 0.562 -----
progupd: 25; m(m(100, 25), 1)
... progress: 7500 (0.0004)
... progress: 7525 (0.0104)
... progress: 7550 (0.0204)
... progress: 7575 (0.0304)
... progress: 7600 (0.0404)
... progress: 7625 (0.0504)
... progress: 7650 (0.0604)
... progress: 7675 (0.0704)
... progress: 7700 (0.0804)
... progress: 7725 (0.0904)
... progress: 7750 (0.1004)
... progress: 7775 (0.1104)
... progress: 7800 (0.1204)
... progress: 7825 (0.1304)
... progress: 7850 (0.1404)
... progress: 7875 (0.1504)
... progress: 790